In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

from pyoperators import MPI

from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions
from qubic.lib.Qsamplings import get_pointing, equ2gal
from qubic.lib.QskySim import Qubic_sky
from qubic.lib.MapMaking.FrequencyMapMaking.Qspectra_component import CMBModel

In [ ]:
%matplotlib inline

In [ ]:
nside = 128

# Initialize QUBIC instance

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
def get_dict(comm, key='in'):
    """QUBIC dictionary.

    Method to modify the qubic dictionary.

    Parameters
    ----------
    key : str, optional
        Can be "in" or "out".
        It is used to build respectively the instances to generate the TODs or to reconstruct the sky maps,
        by default "in".

    Returns
    -------
    dict_qubic: dict
        Modified QUBIC dictionary.

    """

    args = {
        "npointings": 30,
        "nf_recon": 2,
        "nf_sub": 10,
        "nside": nside,
        "MultiBand": True,
        "period": 1,
        "RA_center": 0,
        "DEC_center": -57,
        "filter_nu": 150 * 1e9,
        "noiseless": True,
        "comm": comm,
        "dtheta": 15,
        "nprocs_sampling": 1,
        "nprocs_instrument": comm.Get_size(),
        "photon_noise": False,
        "nhwp_angles": 3,
        "effective_duration150": 3,
        "effective_duration220": 3,
        "filter_relative_bandwidth": 0.25,
        "type_instrument": "two",
        "TemperatureAtmosphere150": None,
        "TemperatureAtmosphere220": None,
        "EmissivityAtmosphere150": None,
        "EmissivityAtmosphere220": None,
        "detector_nep": float(4.7e-17),
        "synthbeam_kmax": 1,
        "synthbeam_fraction": 1,
        "beam_shape": "gaussian",
        "debug":False,
        "kind": "IQU",
    }

    ### Get the default dictionary
    dictfilename = "dicts/pipeline_demo.dict"
    dict_qubic = qubicDict()
    dict_qubic.read_from_file(dictfilename)

    for i in args.keys():

        dict_qubic[str(i)] = args[i]

    return dict_qubic

In [ ]:
# Build MPI communicator and qubic dict
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'
d = qubicDict()
d.read_from_file(dictfilename)

center = equ2gal(d['RA_center'], d['DEC_center'])

d['nf_recon'] = 2
d['MultiBand'] = True
d['nside'] = nside

d['npointings'] = 20
d['synthbeam_kmax'] = 1
nf_sub = d['nf_sub']
d['synthbeam_fraction'] = 1

d['noiseless'] = True
d['photon_noise'] = False
d['use_synthbeam_fits_file'] = False

# comm = MPI.COMM_WORLD
# d["comm"] = comm
# d["nprocs_sampling"] = 1
# d["nprocs_instrument"] = comm.Get_size()
# print("comm.Get_size()", comm.Get_size())
# d = get_dict(comm)

nprocs_instrument = d['nprocs_instrument']
nprocs_sampling = d['nprocs_sampling']
comm = d['comm']

print('------------------------------------')
print('Before')
print('------------------------------------')
print("nprocs_instrument", nprocs_instrument)
print("nprocs_sampling", nprocs_sampling)
print("comm", comm)

if comm is None:
    comm = MPI.COMM_WORLD
if nprocs_instrument is None and nprocs_sampling is None:
    nprocs_instrument = 1
    nprocs_sampling = comm.size
elif nprocs_instrument is None:
    if nprocs_sampling < 1 or nprocs_sampling > comm.size:
        raise ValueError(
            f"Invalid value for nprocs_sampling '{nprocs_sampling}'."
        )
    nprocs_instrument = comm.size // nprocs_sampling
else:
    if nprocs_instrument < 1 or nprocs_instrument > comm.size:
        raise ValueError(
            f"Invalid value for nprocs_instrument '{nprocs_instrument}'."
        )
    nprocs_sampling = comm.size // nprocs_instrument
if nprocs_instrument * nprocs_sampling != comm.size:
    raise ValueError('Invalid MPI distribution of the acquisition.')

d["comm"] = comm
d["nprocs_sampling"] = nprocs_sampling
d["nprocs_instrument"] = nprocs_instrument

print('------------------------------------')
print('After')
print('------------------------------------')
print("nprocs_instrument", d['nprocs_instrument'])
print("nprocs_sampling", d['nprocs_sampling'])
print("comm", d['comm'])

In [ ]:
d

In [ ]:
# Build scanning strategy
sampling = get_pointing(d)

In [ ]:
# Number of sub-acquisitions
nsub_max = 42
nsub_list = np.arange(4, nsub_max + 1, 4)

In [ ]:
TOD_list = []
freq_list = []
maps_list = []

np.random.seed(d['seed'])

# Build CMB map
cl_cmb = CMBModel(None).give_cl_cmb(r=0, Alens=1)
cmb_map = hp.synfast(cl_cmb, nside, new=True, verbose=False).T

for insub in nsub_list:
    input_maps = np.ones((insub, hp.nside2npix(nside), 3)) * cmb_map[None, ...]
    
    # Build acquisition operator
    Qacq = QubicMultiAcquisitions(d, insub, 2, sampling=sampling)
    
    # Build TOD
    tod = []
    map_list = []
    for i in range(insub):
        tod.append(Qacq.H[i](input_maps[i]))
        map_list.append(Qacq.H[i].T(tod[-1]))

    TOD_list.append(np.sum(tod, axis=0))
    maps_list.append(np.mean(map_list, axis=0))
    freq_list.append(np.array(Qacq.allnus))

In [ ]:
TOD_list[0].shape

In [ ]:
for i in range(nsub_list.size):
    plt.plot(np.mean(TOD_list[i], axis=0), label=f"Nsub = {nsub_list[i]}")

plt.title("Mean over detector")
plt.xlabel("Pointing")
plt.ylabel("TOD")
plt.legend(loc='best')

In [ ]:
det_index = 200

for i in range(nsub_list.size):
    plt.plot(TOD_list[i][det_index], label=f"Nsub = {nsub_list[i]}")

plt.title(f"Detector {det_index}")
plt.xlabel("Pointing")
plt.ylabel("TOD")
plt.legend(loc='best')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

for i in range(len(TOD_list)):
    rel_error = np.mean(np.abs((TOD_list[i] - TOD_list[-1]) / TOD_list[-1]))
    
    ax1.plot(nsub_list[i], rel_error, '.')
    ax2.plot(nsub_list[i], rel_error, '.')

ax1.set_xlabel('Nsub')
ax1.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax1.set_title('Convergence of H (linear scale)')

ax2.set_xlabel('Nsub')
ax2.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax2.set_title('Convergence of H (log scale)')
ax2.set_yscale('log')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

for i in range(len(TOD_list)):
    rel_error = np.mean(np.abs((TOD_list[-1][:, 0] - TOD_list[i][:, 0]) / TOD_list[-1][:, 0]))
    
    ax1.plot(nsub_list[i], rel_error, '.')
    ax2.plot(nsub_list[i], rel_error, '.')

ax1.set_xlabel('Nsub')
ax1.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax1.set_title('Convergence of H (linear scale)')

ax2.set_xlabel('Nsub')
ax2.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax2.set_title('Convergence of H (log scale)')
ax2.set_yscale('log')